In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report

# Define ImageDataGenerators for training and validation with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=[0.2, 1.5]
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\train",  # Replace with your train directory path
    target_size=(299, 299),  # InceptionV3 input size
    batch_size=32,
    class_mode='binary'  # Two classes: ALS and Healthy
)

val_generator = val_datagen.flow_from_directory(
    r"C:\Users\Priyamvadha Pradeep\Desktop\FYP\CNN\New Models\validate",  # Replace with your validation directory path
    target_size=(299, 299),  # InceptionV3 input size
    batch_size=32,
    class_mode='binary'
)

# Load pre-trained InceptionV3 model with ImageNet weights (not including the top layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create the model by adding custom layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())  # For dimensionality reduction
model.add(Dropout(0.5))  # Add dropout to reduce overfitting
model.add(Dense(256, activation='relu'))  # Dense layer
model.add(Dense(1, activation='sigmoid'))  # Binary classification output

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_generator,
    epochs=40,  # You can adjust this number
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on validation data
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Get predictions for confusion matrix
y_pred = model.predict(val_generator)
y_pred = (y_pred > 0.5)  # Convert probabilities to class labels

# True labels
y_true = val_generator.classes

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
report = classification_report(y_true, y_pred, target_names=train_generator.class_indices.keys())
print("Classification Report:")
print(report)

Found 978 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/40


c:\Users\Priyamvadha Pradeep\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


31/31 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.6050 - loss: 0.7178 - val_accuracy: 0.6667 - val_loss: 0.6416 - learning_rate: 1.0000e-04
Epoch 2/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.6652 - loss: 0.6439 - val_accuracy: 0.6667 - val_loss: 0.6345 - learning_rate: 1.0000e-04
Epoch 3/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.6380 - loss: 0.6627 - val_accuracy: 0.6707 - val_loss: 0.6322 - learning_rate: 1.0000e-04
Epoch 4/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.6374 - loss: 0.6285 - val_accuracy: 0.6748 - val_loss: 0.6327 - learning_rate: 1.0000e-04
Epoch 5/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.6623 - loss: 0.6405 - val_accuracy: 0.6748 - val_loss: 0.6309 - learning_rate: 1.0000e-04
Epoch 6/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.6414 - loss: 0.6596 - val_accuracy: 0.6748 - val_loss: 0.6273 - learning_rate: 1.0000e-04
Epoch 7/40
31/31 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.6208 - loss: 0.6515 - v